# Gaussova eliminacija

## Općenito

Sustav $Ax=b$ se rješava u tri koraka (__bez pivotiranja__)

1. $A=LU$ (LU rastav, $O(\frac{2}{3}n^3)$ operacija)
2. $Ly=b$ (donje trokutrasti sustav, $n^2$ operacija)
3. $Ux=y$ (gornje torkutasti sustav, $n^2$ operacija)

Ukoliko pivotiramo, tada je

1. $PA=LU$ 
2. $Ly=P^T b$
3. $Ux=y$ 

## LU rastav

In [1]:
function mylu{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    U=map(Float64,[zero(A[1,1]) for i=1:n, j=1:n]) # This acccepts blocks and numbers
    L=map(Float64,[zero(A[1,1]) for i=1:n, j=1:n])
    for k=1:n
        L[k,k]=one(A[1,1])
        for i=k+1:n
            L[i,k]=A[i,k]/A[k,k]
            for j=k+1:n
                A[i,j]=A[i,j]-L[i,k]*A[k,j]
            end
        end
        for j=k:n
            U[k,j]=A[k,j]
        end
    end
    L,U
end

mylu (generic function with 1 method)

In [2]:
A=rand(6,6)
# A=[2.0 2;3 4]

6x6 Array{Float64,2}:
 0.0286382  0.996928   0.419462   0.851321   0.376926  0.306283 
 0.491405   0.266544   0.656151   0.413214   0.934482  0.379227 
 0.793622   0.690088   0.334575   0.0426215  0.681999  0.526546 
 0.983122   0.0377224  0.639114   0.965501   0.808232  0.812517 
 0.277411   0.64395    0.832936   0.313845   0.497412  0.0829397
 0.228316   0.0805823  0.0929685  0.379679   0.764184  0.807339 

In [3]:
L,U=mylu(A)

(
6x6 Array{Float64,2}:
  1.0      0.0        0.0        0.0        0.0      0.0
 17.1591   1.0        0.0        0.0        0.0      0.0
 27.712    1.59959    1.0        0.0        0.0      0.0
 34.329    2.03006    0.582492   1.0        0.0      0.0
  9.68675  0.535223  -0.327895  -0.584029   1.0      0.0
  7.97243  0.467188   0.236195   0.404056  -1.00315  1.0,

6x6 Array{Float64,2}:
 0.0286382    0.996928   0.419462    0.851321   0.376926   0.306283
 0.0        -16.8398    -6.54142   -14.1946    -5.53322   -4.8763  
 0.0          0.0       -0.825962   -0.843559  -0.912501  -0.161085
 0.0          0.0        0.0         1.04787   -0.36698    0.291144
 0.0          0.0        0.0         0.0       -0.705807  -0.156823
 0.0          0.0        0.0         0.0        0.0        0.406761)

In [4]:
L*U-A

6x6 Array{Float64,2}:
 0.0   0.0           0.0           0.0           0.0           0.0        
 0.0  -1.11022e-15   2.22045e-16   0.0          -4.44089e-16  -4.44089e-16
 0.0   6.66134e-16  -2.22045e-16  -1.33227e-15  -4.44089e-16   4.44089e-16
 0.0  -9.32587e-15  -4.44089e-16  -1.11022e-15   1.11022e-16   2.22045e-16
 0.0  -8.88178e-16  -2.22045e-16   2.22045e-16   1.11022e-16   0.0        
 0.0   2.22045e-16   0.0           0.0           0.0           1.11022e-16

## Trokutasti sustavi

In [5]:
function myU{T}(U::Array{T},b1::Array{T})
    b=deepcopy(b1)
    n=length(b)
    for i=n:-1:1
       for j=n:-1:i+1
            b[i]=b[i]-U[i,j]*b[j]
       end
        b[i]=b[i]/U[i,i]
    end
    b
end

function myL{T}(L::Array{T},b1::Array{T})
    b=deepcopy(b1)
    n=length(b)
    for i=1:n
        for j=1:i-1
            b[i]=b[i]-L[i,j]*b[j]
        end
        b[i]=b[i]/L[i,i]
    end
    b
end

myL (generic function with 1 method)

In [6]:
b=rand(6)

6-element Array{Float64,1}:
 0.00973218
 0.641216  
 0.838202  
 0.00588544
 0.196349  
 0.578017  

In [7]:
x=A\b

6-element Array{Float64,1}:
  0.0725106
  0.333818 
 -0.59867  
 -0.585785 
  1.3375   
 -0.259456 

In [8]:
y=myL(L,b)

6-element Array{Float64,1}:
  0.00973218
  0.474221  
 -0.190055  
 -1.1802    
 -0.903331  
 -0.105537  

In [9]:
x1=myU(U,y)

6-element Array{Float64,1}:
  0.0725106
  0.333818 
 -0.59867  
 -0.585785 
  1.3375   
 -0.259456 

In [10]:
x-x1

6-element Array{Float64,1}:
 -3.13638e-15
 -2.77556e-16
  1.9984e-15 
  4.44089e-16
 -6.43929e-15
  4.94049e-15

## Brzina

Program `mylu()` je jako spor. Između ostalog, alocira nepotrebno tri matrice.

Program se može reformulirati na načun da su i $L$ i $U$ spremljene u polje $A$, pri čemu se dijagonala od $L$ ne sprema, jer su svi elementi jednaki 1.

In [11]:
function mylu1{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    for k=1:n-1
        rho=k+1:n
        A[rho,k]=A[rho,k]/A[k,k]
        A[rho,rho]=A[rho,rho]-A[rho,k]*A[k,rho]
    end
    A
end

mylu1 (generic function with 1 method)

In [12]:
mylu1(A)

6x6 Array{Float64,2}:
  0.0286382    0.996928   0.419462    0.851321   0.376926   0.306283
 17.1591     -16.8398    -6.54142   -14.1946    -5.53322   -4.8763  
 27.712        1.59959   -0.825962   -0.843559  -0.912501  -0.161085
 34.329        2.03006    0.582492    1.04787   -0.36698    0.291144
  9.68675      0.535223  -0.327895   -0.584029  -0.705807  -0.156823
  7.97243      0.467188   0.236195    0.404056  -1.00315    0.406761

In [13]:
L,U

(
6x6 Array{Float64,2}:
  1.0      0.0        0.0        0.0        0.0      0.0
 17.1591   1.0        0.0        0.0        0.0      0.0
 27.712    1.59959    1.0        0.0        0.0      0.0
 34.329    2.03006    0.582492   1.0        0.0      0.0
  9.68675  0.535223  -0.327895  -0.584029   1.0      0.0
  7.97243  0.467188   0.236195   0.404056  -1.00315  1.0,

6x6 Array{Float64,2}:
 0.0286382    0.996928   0.419462    0.851321   0.376926   0.306283
 0.0        -16.8398    -6.54142   -14.1946    -5.53322   -4.8763  
 0.0          0.0       -0.825962   -0.843559  -0.912501  -0.161085
 0.0          0.0        0.0         1.04787   -0.36698    0.291144
 0.0          0.0        0.0         0.0       -0.705807  -0.156823
 0.0          0.0        0.0         0.0        0.0        0.406761)

Usporedimo brzine LAPACK-ovog programa `lu()` i našeg naivnog programa `mylu()`na većoj dimenziji. 

Izvedite program par puta radi točnijeg mjerenja brzine.

In [14]:
n=512
A=rand(n,n)

512x512 Array{Float64,2}:
 0.420964   0.842604    0.00987771  …  0.0377011   0.850471   0.515657 
 0.34277    0.217502    0.806407       0.25799     0.163236   0.69811  
 0.102501   0.00454236  0.819973       0.903898    0.33709    0.605441 
 0.717142   0.560323    0.0689171      0.296568    0.234182   0.974818 
 0.970251   0.152696    0.620573       0.284084    0.0632172  0.796746 
 0.757579   0.499103    0.56142     …  0.321354    0.687077   0.0387579
 0.220312   0.531842    0.928252       0.777819    0.422078   0.406508 
 0.998365   0.652645    0.677373       0.252459    0.783683   0.577668 
 0.139444   0.129599    0.1195         0.242708    0.805566   0.952466 
 0.663471   0.287256    0.592841       0.824083    0.627295   0.544591 
 0.177768   0.959652    0.776663    …  0.95663     0.584324   0.901959 
 0.127012   0.110917    0.0487915      0.398639    0.418546   0.168855 
 0.8671     0.804874    0.605825       0.329196    0.444365   0.272595 
 ⋮                                  ⋱ 

In [15]:
@time lu(A);

In [16]:
@time mylu1(A);

0.156163 seconds (177.58 k allocations: 14.420 MB, 2.91% gc time)
  

### Blok varijanta

`mylu()` je nekoliko desetaka puta sporiji. `mylu1()` je malo sporiji.

Probajmo s blokovima:

Preradimo `mylu1()` za rad s blokovima (nemamo ugrađeno pivotiranje!)

In [17]:
function mylu2{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    for k=1:n-1
        for rho=k+1:n
            A[rho,k]=A[rho,k]/A[k,k]
            for l=k+1:n
                A[rho,l]=A[rho,l]-A[rho,k]*A[k,l]
            end
        end
    end
    A
end

mylu2 (generic function with 1 method)

Napravimo prvo mali test:

In [18]:
# Probajte k,l=32,16 i k,l=64,8
k,l=4,4
Ab=[rand(k,k) for i=1:l, j=1:l];

In [19]:
A0=mylu2(Ab)

4x4 Array{Any,2}:
 4x4 Array{Float64,2}:
 0.980802  0.0139078  0.431553   0.539358
 0.904431  0.395608   0.0848852  0.683929
 0.807037  0.203334   0.708745   0.323855
 0.222566  0.540591   0.760389   0.232899              …  4x4 Array{Float64,2}:
 0.512     0.0663652  0.883554  0.913826
 0.57078   0.225595   0.356429  0.86303 
 0.995059  0.248058   0.462482  0.2388  
 0.875004  0.544303   0.871858  0.585355                
 4x4 Array{Float64,2}:
  0.230729   0.639786  -0.103634    0.85274 
 -1.8426     0.814709   2.71627    -1.28311 
 -0.0203742  0.329107  -0.0236792   0.519537
  0.934637   1.13836   -2.07375     1.58295      4x4 Array{Float64,2}:
 -0.367953  -0.0941262  -1.08854  -0.32616 
 -0.27383    0.81178     2.13082   1.09918 
 -0.140088  -0.282747    0.29729   0.346121
  0.239581   0.0351531  -1.47019  -2.23079     
 4x4 Array{Float64,2}:
  2.04505    -0.253828  -2.35392    1.7226  
  3.95954    -0.502491  -5.03649    3.11375 
 -0.467458    0.609575   0.979375  -0.178589
  0.05

0.404228 seconds (17.26 k allocations: 1.004 GB, 9.24% gc time)


In [20]:
# Provjera
U=triu(A0)
L=tril(A0)
for i=1:maximum(size(L))
    L[i,i]=eye(L[1,1])
end

In [21]:
Res=L*U-Ab

4x4 Array{Any,2}:
 4x4 Array{Float64,2}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0                                                                                                                                              …  4x4 Array{Float64,2}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0                                                                                                                                            
 4x4 Array{Float64,2}:
 0.0  1.11022e-16   0.0          -1.11022e-16
 0.0  1.11022e-16  -3.33067e-16  -5.55112e-17
 0.0  0.0           5.55112e-17   0.0        
 0.0  0.0          -1.11022e-16   0.0                                                 4x4 Array{Float64,2}:
 0.0  0.0   0.0          0.0
 0.0  0.0  -2.22045e-16  0.0
 0.0  0.0   0.0          0.0
 0.0  0.0   0.0          0.0                                                                                                        
 4x4

In [22]:
# pretvaranje blok matrice u obicnu
unblock(A) = mapreduce(identity, hcat, [mapreduce(identity, vcat, A[:,i]) for i = 1:size(A,2)])

unblock (generic function with 1 method)

In [23]:
norm(unblock(Res))

1.2223255408216555e-15

Sada probajmo veću dimenziju:

In [24]:
# Probajmo vece dimenzije (n=k*l)
k,l=32,16
Ab=[rand(k,k) for i=1:l, j=1:l];

In [25]:
@time mylu2(Ab);

Vidimo da je `mylu2()` gotovo jednako brz kao `lu()`, uz napomenu da `mylu2()` nema ugrađeno pivotiranje. 

## Pivotiranje

Stanardne implementacije uvijek računaju Gaussovu eliminaciju s _parcijalnim pivotiranjem_:

> u svakom koraku se retci pivotiranju tako da pivotni element ima najveću apsolutnu vrijednst u danom stupcu. Na taj 
> način je 
> 
> $$|L_{ij}| \leq 1,$$
> što u praksi dovoljno spriječava rast elemenata.


In [26]:
A=rand(5,5)
L,U,P=lu(A)

(
5x5 Array{Float64,2}:
 1.0         0.0        0.0         0.0       0.0
 0.0477668   1.0        0.0         0.0       0.0
 0.766207   -0.871317   1.0         0.0       0.0
 0.927159    0.113133  -0.216876    1.0       0.0
 0.887795    0.515538  -0.0408529  -0.219176  1.0,

5x5 Array{Float64,2}:
 0.764691  0.753711  0.369407  0.236753  0.0825824
 0.0       0.611039  0.817107  0.720657  0.196987 
 0.0       0.0       1.13232   0.981444  0.616188 
 0.0       0.0       0.0       0.569025  0.775278 
 0.0       0.0       0.0       0.0       0.276186 ,

[5,4,1,3,2])

In [27]:
L*U-A[P,:]

0.057176 seconds (8.79 k allocations: 26.526 MB, 6.73% gc time)


5x5 Array{Float64,2}:
 0.0  0.0   0.0          0.0  0.0
 0.0  0.0   0.0          0.0  0.0
 0.0  0.0  -1.11022e-16  0.0  0.0
 0.0  0.0   0.0          0.0  0.0
 0.0  0.0   0.0          0.0  0.0

### Potpuno pivotiranje

Sljedeći program računa Gaussovu eliminaciju s _potpunim pivotiranjem_ - u svakom koraku retci i stupci zamijene na način da se na pivotnu poziciju dovede element koji ima najveću apsolutnu vrijednost u trenutnoj podmatrici.

In [28]:
function gecp{T}(A1::Array{T})
    # Gaussova eliminacija s potpunim pivotiranjem
    # Izlaz: Pr*L*U*Pc'=A ili Pr'*A*Pc=L*U
    A=deepcopy(A1)
    n,m=size(A)
    Pr=eye(n,n)
    Pc=eye(n,n)
    D=zeros(n)
    for i=1:n-1
        am1,im1=findmax(abs(A[i:n,i:n]),1)
        am,JJ=findmax(am1)
        II=mod(im1[JJ],(n-i+1))
        if II==0
            II=n-i+1
        end
        imax=II+i-1
        jmax=JJ+i-1
        #  zamijena redaka
        if (imax != i)
            temp = Pr[:,i]
            Pr[:,i] = Pr[:,imax]
            Pr[:,imax] = temp
            temp = A[i,:]
            A[i,:] = A[imax,:]
            A[imax,:] = temp
        end
        # zamijena stupaca
        if (jmax != i)
            temp = Pc[:,i]
            Pc[:,i] = Pc[:,jmax]
            Pc[:,jmax] = temp
            temp = A[:,i]
            A[:,i] = A[:,jmax]
            A[:,jmax] = temp
        end
        # eliminacija
        D[i]=A[i,i]
        A[i+1:n,i] = A[i+1:n,i]/D[i]
        A[i+1:n,i+1:n] = A[i+1:n,i+1:n] - A[i+1:n,i]*A[i,i+1:n]
        A[i,i+1:n]=A[i,i+1:n]/D[i]
    end
    D[n]=A[n,n]
    L=eye(n,n)+tril(A,-1)
    U=eye(n,n)+triu(A,1)
    U=diagm(D)*U
    L,U,Pr,Pc
end

gecp (generic function with 1 method)

In [29]:
n=5
A=rand(n,n)
b=rand(n)

5-element Array{Float64,1}:
 0.25267 
 0.812403
 0.815209
 0.457422
 0.358541

In [30]:
L,U,Pr,Pc=gecp(A)

(
5x5 Array{Float64,2}:
 1.0        0.0       0.0       0.0       0.0
 0.0351423  1.0       0.0       0.0       0.0
 0.692114   0.731753  1.0       0.0       0.0
 0.320083   0.479005  0.28299   1.0       0.0
 0.32043    0.58712   0.41791  -0.204042  1.0,

5x5 Array{Float64,2}:
 0.907666  0.418957   0.61279    0.341787   0.153519 
 0.0       0.679073   0.5851     0.469467   0.187559 
 0.0       0.0       -0.594571  -0.216837   0.504112 
 0.0       0.0        0.0        0.446352  -0.0220349
 0.0       0.0        0.0        0.0       -0.236912 ,

5x5 Array{Float64,2}:
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0
 0.0  1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0,

5x5 Array{Float64,2}:
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0)

In [31]:
Pr*L*U*Pc'-A

5x5 Array{Float64,2}:
 0.0  0.0  0.0   0.0          0.0
 0.0  0.0  0.0   5.55112e-17  0.0
 0.0  0.0  0.0  -2.77556e-17  0.0
 0.0  0.0  0.0   0.0          0.0
 0.0  0.0  0.0   0.0          0.0

In [32]:
y=myL(L,Pr'*b)

5-element Array{Float64,1}:
  0.457422 
  0.799134 
 -0.648688 
  0.0129112
  0.470371 

In [33]:
z=myU(U,y)

5-element Array{Float64,1}:
  0.204772 
  2.2616   
 -0.567144 
 -0.0690878
 -1.98542  

In [34]:
x=Pc*z

5-element Array{Float64,1}:
  2.2616   
 -0.0690878
 -0.567144 
 -1.98542  
  0.204772 

In [35]:
A*x-b

5-element Array{Float64,1}:
 -2.22045e-16
 -1.11022e-16
 -2.22045e-16
  5.55112e-17
  0.0        

## Točnost

Neka je zadan sustav $Ax=b$, pri čemu je matrica $A$ regularna.

Da bi primijenili koncepte iz bilježnice [NA04_Pogreska_unatrag_i stabilni_algoritmi](NA04_Pogreska_unatrag_i_stabilni_algoritmi.ipynb), potrebno je:

1. napraviti teoriju smetnje za dani problem
2. analizirati pogreške algoritma (Gaussove eliminacije)

### Teorija smetnje

Neka je 

$$
(A+\delta A)\hat x=(b+\delta b)
$$
za neki $\hat x=x+\delta x$.

Želimo ocijeniti 
$$
\frac{\| \hat x - x \|}{\| x\|} \equiv \frac{\| \delta x\|}{\| x\|}.
$$

Uvedimo oznake (npr. prema [G. Golub and C. F. Van Loan, Matrix Computations, str. 87, poglavlje 2.6.2][GVL13])

$$
\delta A=\varepsilon F, \quad \delta b=\varepsilon f, \qquad \hat x=x(\varepsilon),
$$
čime smo dobili jednodimenzionalni problem 

$$
(A+\varepsilon F)x(\varepsilon)=b+\varepsilon f.
$$

za neke (nepoznate) matricu $F$ i vektor $f$. 

Deriviranje po $\varepsilon$ daje

$$
Fx(\varepsilon)+(A+\varepsilon F)\dot x(\varepsilon)=f.
$$

Uvrštavanje $\varepsilon=0$ daje

$$
F x+A\dot x(0)=f,
$$
odnosno
$$
\dot x(0)=A^{-1}(f-Fx).
$$

Taylorov razvoj oko $x(0)$ glasi

$$
x(\varepsilon)=x(0)+\varepsilon \dot x(0) +O(\varepsilon^2),
$$
odnosno, uz zanemarivanje člana $O(\varepsilon^2)$,

$$
\hat x-x=\varepsilon A^{-1}(f-Fx)=A^{-1} (\varepsilon f + \varepsilon F x) = A^{-1} (\delta b + \delta A x).
$$

Svojstva norme povlače

$$
\| \hat x-x\|\leq \| A^{-1} \| (\| \delta b \|  + \| \delta A \| \cdot \|  x\| ).
$$

Konačno, zbog $\| b\| \leq \| A\| \| x\|$ imamo

$$
\frac{\| \hat x-x\|}{\| x\|}\leq \| A\|  \cdot \| A^{-1} \| \bigg(\frac{\| \delta b \|}{\|b\|}  + \frac{\| \delta A \|}{ \|  A\|} \bigg). \tag{1}
$$

Broj 
$$
\kappa(A)\equiv \| A\|  \cdot \| A^{-1} \|
$$ 

je _uvjetovanost_ (_kondicija_)  matrice $A$ i kazuje nam 

> koliko se relativno uvećaju relativne promjene u polaznim podacima (matrici $A$ i vektoru $b$).

Pogledajmo primjer iz [R. Scitovski, Numerička matematika, str. 42][RS04]:




[GVL13]: https://books.google.hr/books?id=X5YfsuCWpxMC&printsec=frontcover&hl=hr#v=onepage&q&f=false "G. Golub and C. F Van Loan, 'Matrix Computations', 4th Edition, John Hopkins, Baltimore, 2013" 

[RS04]: http://www.mathos.unios.hr/pim/Materijali/Num.pdf "R. Scitovski, 'Numerička matematika', Sveučilište u Osijeku, osijek, 2004."

In [36]:
A= [0.234 0.458; 0.383 0.750]

2x2 Array{Float64,2}:
 0.234  0.458
 0.383  0.75 

In [37]:
b=[0.224;0.367]

2-element Array{Float64,1}:
 0.224
 0.367

In [38]:
x=A\b

2-element Array{Float64,1}:
 -1.0
  1.0

In [39]:
δb=[0.00009; 0.000005]
x1=A\(b+δb)

2-element Array{Float64,1}:
 -0.241744
  0.612791

In [40]:
cond(A), norm(δb)/norm(b), norm(x1-x)/norm(x)

(11322.197586092605,0.0002096449170953002,0.6020311134825742)

In [41]:
δA=[-0.001 0;0 0]
x2=(A+δA)\b

2-element Array{Float64,1}:
 0.129518
 0.423193

In [42]:
cond(A), norm(δA)/norm(A), norm(x2-x)/norm(x)

(11322.197586092605,0.0010134105230118603,0.896804787832142)

### Pogreška Gaussove eliminacije

Prema [G. Golub and C. F. Van Loan, Matrix Computations, str. 122, poglavlje 3.3][GVL13], za izračunate faktore
$\hat L$ i $\hat U$ vrijedi

$$
\hat L\cdot \hat U = A+\delta A
$$

gdje je (nejednakost se čita po elementima matrica, $\varepsilon$ je sada točnost stroja)

$$
| \delta A|\leq 3(n-1) \varepsilon (|A|+|\hat L| \cdot |\hat U|) +O(\varepsilon^2).
$$

Zanemarivanje člana $O(\varepsilon^2)$ i prelazak na normu daju

$$
\|\delta A \| \approx \leq  O(n)\varepsilon (\| A\| + \| \hat L\| \cdot \| \hat U\|),
$$

pa je 

$$
 \frac{\|\delta A \|}{\|A\|} \leq O(n)\varepsilon \bigg(1+\frac{\| \hat L\| \cdot \| \hat U\|}{\|A\|}\bigg).
$$

Ukoliko se Gaussova eliminacija radi s pivotiranjem, tada će najvjerojatnije zadnji kvocijent također biti malen 
($\approx 1$). Također, pogreška kod rješavanja trokutastih sustava nije veća od navedene pa, uvrštavanjme u (1), slijedi 
da za relativnu pogrešku izračunatog rješenja vrijedi

$$
\frac{\| \hat x-x\|}{\| x\|}\leq \kappa(A) O(n\varepsilon).
$$

> __Ukoliko je kondicija matrice velika, rješenje može biti netočno.__

[GVL13]: https://books.google.hr/books?id=X5YfsuCWpxMC&printsec=frontcover&hl=hr#v=onepage&q&f=false "G. Golub and C. F Van Loan, 'Matrix Computations', 4th Edition, John Hopkins, Baltimore, 2013" 

In [43]:
n=10
v=rand(n)

10-element Array{Float64,1}:
 0.797814 
 0.444895 
 0.709601 
 0.597694 
 0.705077 
 0.361814 
 0.360566 
 0.0324751
 0.922115 
 0.533028 

In [44]:
A=Array(Float64,n,n)
for i=1:n
    A[:,i]=v.^(i-1)
end
A=A'

10x10 Array{Float64,2}:
 1.0       1.0          1.0        …  1.0          1.0       1.0       
 0.797814  0.444895     0.709601      0.0324751    0.922115  0.533028  
 0.636507  0.197931     0.503533      0.00105463   0.850296  0.284119  
 0.507814  0.0880587    0.357308      3.42493e-5   0.78407   0.151444  
 0.405141  0.0391769    0.253546      1.11225e-6   0.723003  0.0807237 
 0.323227  0.0174296    0.179916   …  3.61205e-8   0.666692  0.043028  
 0.257875  0.00775433   0.127669      1.17302e-9   0.614767  0.0229352 
 0.205736  0.00344986   0.0905938     3.80939e-11  0.566885  0.0122251 
 0.164139  0.00153483   0.0642854     1.2371e-12   0.522733  0.00651632
 0.130953  0.000682837  0.045617      4.01751e-14  0.48202   0.00347338

In [45]:
b=rand(n)

10-element Array{Float64,1}:
 0.42553 
 0.371556
 0.148544
 0.860282
 0.905312
 0.320869
 0.738031
 0.97861 
 0.249419
 0.371056

In [46]:
x=A\b

10-element Array{Float64,1}:
     -2.35296e5
 -76790.8      
      1.45411e7
      1.36093e6
     -1.50267e7
      7.18195e6
     -7.02774e6
    442.539    
   8121.48     
     -7.26e5   

In [47]:
cond(A)

4.406777549324911e9

In [48]:
Ab=Array(BigFloat,n,n)
bb=Array(BigFloat,n)
for i=1:n
    for j=1:n
        Ab[i,j]=convert(BigFloat,A[i,j])
    end
    bb[i]=convert(BigFloat,b[i])
end
xb=Ab\bb

10-element Array{BigFloat,1}:
 -2.352962154099944677391051554557723188000947314788677695782357969102015674385169e+05
 -7.679082279698689397743472191244564092552794805157778301136802860956015994332717e+04
  1.454112671268589420111460025315177277958333835971048593165989887106462236946462e+07
  1.360933484061750177750226027753807363448904005726737933642085260271461118693593e+06
 -1.502674350543754528633033745322596020035683598832053399921036971025125871474275e+07
  7.181949768330400080429285979976183043561292783866435060046161786486792298948892e+06
 -7.027743232595135731092502637368001177216286980060280339288289626039988531577596e+06
  4.425386214536011051316283906438406001327557944473881380280867406938887307583459e+02
  8.121475867804788956539731663493749122877931725744566408932893638864623877555401e+03
 -7.259997777973229163419105668324928795908368595974054419318437363914253260132919e+05

In [49]:
norm(xb-x)/norm(xb)

1.11676516081762812644413651825164845797768932886252142021092364910332110193646e-08

### Umjetno loša kondicija

In [50]:
A=[1 1; 1 2]
b=[1;3]
x=A\b
@show x,cond(A)
A1=[1e-4 1e-4;1 2]
b1=[1e-4;3]
x1=A1\b1
x,cond(A1),x-x1

(x,cond(A)) = ([-1.0,2.0],6.854101966249685)

([-1.0,2.0],50000.00017991671,[8.881784197001252e-16,-4.440892098500626e-16])

## Rezidual

(Prema [G. W. Stewart, Afternotes on Numerical Analysis, str. 128.][Ste96])

Izračunato rješenje $\hat x$ sustava $Ax=b$ je točno rješenje nekog sličnog sustava

$$ 
(A+\delta A)\hat x=b. \tag{1}
$$

_Rezidual_ (ili _ostatak_) definiramo kao 

$$
r=b-A\hat x.
$$

Tada je 

$$
0=b-(A+\delta A)\hat x=r- \delta A\hat x
$$
pa je 

$$ 
\| r\| \leq \| \delta A\hat x \| \leq \| \delta A\| \cdot \|\hat x \|,
$$

odnosno

$$
\frac{\|  \delta A\|}{\|A \|} \geq \frac{\|r\|}{\| A\| \cdot \|\hat x \|}.
$$

Dakle,
> ako  _relativni rezidual_ 
>
> $$ \frac{r}{\| A\| \cdot \|\hat x \|}$$
ima veliku normu, tada __rješenje nije izračunato stabilno.__

S druge strane, ako relativni rezidual ima malu normu, tada je rješenje izračunato stabilno. Naime, za

$$
\delta A=\frac{r\hat x^T}{\|\hat x\|^2}
$$
vrijedi (1):

$$
b-(A+\delta A)\hat x=(b-A\hat x)-\delta A \hat x = r-\frac{r\hat x^T \hat x}{\|\hat x\|^2}
= r-\frac{r \|\hat x^T \hat x\|}{\|\hat x\|^2}=r-r=0.
$$
Također vrijedi

$$
\frac{\|  \delta A\|}{\|A \|}  \leq  \frac{\|r\|\|\hat x \|}{\| A\| \cdot \|\hat x \|^2}=
\frac{\|r\|}{\| A\| \cdot \|\hat x \|}.
$$




[Ste96]: https://books.google.hr/books?id=w-2PWh01kWcC&printsec=frontcover&hl=hr#v=onepage&q&f=false    "G. W. Stewart, 'Afternotes on Numerical Analysis', SIAM, Philadelphi, 1996"

In [51]:
r=b-A*x

2-element Array{Float64,1}:
 0.0
 0.0

In [52]:
norm(r)/(norm(A)*norm(x))

0.0